In [1]:
import pandas as pd
import numpy as np
import networkx as nx

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, mean_squared_error
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectFwe, f_classif # f score
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE

In [2]:
atom = pd.read_pickle('atom.pkl')
bond = pd.read_pickle('bond.pkl')
molecule = pd.read_pickle('molecule.pkl')

In [3]:
atom.head()

,atom_id,charge,element,molecule_id,type
0,d100_1,-0.128,c,d100,22
1,d100_10,0.132,h,d100,3
2,d100_11,0.002,c,d100,29
3,d100_12,-0.128,c,d100,22
4,d100_13,-0.128,c,d100,22


In [4]:
bond.head()

,atom1_id,atom2_id,type
0,d100_1,d100_2,7
1,d100_1,d100_7,1
2,d100_11,d100_12,7
3,d100_12,d100_13,7
4,d100_12,d100_17,1


In [5]:
molecule.head()

,ind1,inda,logp,lumo,molecule_id,mutagenic
0,1,0,4.23,-1.246,d1,yes
1,1,0,4.62,-1.387,d10,yes
2,0,0,2.68,-1.034,d100,no
3,1,0,6.26,-1.598,d101,yes
4,1,0,2.40,-3.172,d102,yes


Aim of this dataset is to determine if molecules are mutagenic.

Outline of procedure to apply RELAGG is to:
1. Determine all the pairwise id relationships used to join
2. Determine the star schema which we wish to aggregate
3. Produce a flattened dataset as needed

---

To produce the correct dataset I propose that we:

*  produce pair wise relationship `molecule_id -> atom_id -> [atom1_id, atom2_id] -> atom_id`

In [6]:
m_atom = atom[['molecule_id', 'atom_id']].to_dict(orient='records')
atom_a1 = bond[['atom1_id', 'atom2_id']].rename(columns={'atom2_id':'atom_id'}).to_dict(orient='records')
atom_a2 = bond[['atom1_id', 'atom2_id']].rename(columns={'atom1_id':'atom_id'}).to_dict(orient='records')

# produce a flatten structure with all data

In [7]:
m_atom_pairs = [("molecule_id:{}".format(x['molecule_id']), "atom_id:{}".format(x['atom_id']))
  for x in m_atom
]
atom_a2_pairs = [("atom_id:{}".format(x['atom_id']), "atom_id:{}".format(x['atom2_id']))
  for x in atom_a2
]
atom_a1_pairs = [("atom_id:{}".format(x['atom_id']), "atom_id:{}".format(x['atom1_id']))
  for x in atom_a1
]


In [8]:
all_molecule_id = list(set([x[0] for x in m_atom_pairs]))

G = nx.Graph() # we have cycles in this representation
G.add_nodes_from(all_molecule_id) # orient from entity level of molecule
G.add_nodes_from([x[1] for x in m_atom_pairs]) # add all atoms
G.add_edges_from(m_atom_pairs)
G.add_edges_from(atom_a2_pairs)
G.add_edges_from(atom_a1_pairs)

In [9]:
# To get all atoms connected to molecule we would do this:
list(nx.single_source_shortest_path_length(G, all_molecule_id[0]).keys())

['molecule_id:d128',
 'atom_id:d128_11',
 'atom_id:d128_19',
 'atom_id:d128_7',
 'atom_id:d128_12',
 'atom_id:d128_6',
 'atom_id:d128_5',
 'atom_id:d128_18',
 'atom_id:d128_2',
 'atom_id:d128_16',
 'atom_id:d128_8',
 'atom_id:d128_20',
 'atom_id:d128_1',
 'atom_id:d128_17',
 'atom_id:d128_22',
 'atom_id:d128_10',
 'atom_id:d128_4',
 'atom_id:d128_3',
 'atom_id:d128_13',
 'atom_id:d128_14',
 'atom_id:d128_9',
 'atom_id:d128_21',
 'atom_id:d128_15']

In [10]:
def clean_molecule_atom_rship(ls):
    atoms = [x.split(':')[1] for x in ls if not x.startswith("molecule")]
    molecule = [x.split(':')[1] for x in ls if x.startswith("molecule")][0]
    return (molecule, atoms)

In [11]:
# get all relationships of interest at molecule level
molecule_level_rship = [
    clean_molecule_atom_rship(list(nx.single_source_shortest_path_length(G, x).keys())) for x in 
    all_molecule_id
]
molecule_level_rship[0]

('d128',
 ['d128_11',
  'd128_19',
  'd128_7',
  'd128_12',
  'd128_6',
  'd128_5',
  'd128_18',
  'd128_2',
  'd128_16',
  'd128_8',
  'd128_20',
  'd128_1',
  'd128_17',
  'd128_22',
  'd128_10',
  'd128_4',
  'd128_3',
  'd128_13',
  'd128_14',
  'd128_9',
  'd128_21',
  'd128_15'])

From here we will need to aggregate all this data to a single record. 

Assume columns:
*  `type` are categorical

We will only use te `atom` table as the flat entity table.

In [12]:
def dataframe_info(df, prefix='num'):
    """
    returns a dataframe with a single row, 
    where the column names as `col_{statistic}`
    """
    #df = pd.DataFrame({col: srs})
    def percentile(n):
        def percentile_(x):
            return np.percentile(x, n)
        percentile_.__name__ = 'percentile_%s' % n
        return percentile_
    df_desc = df.groupby([True]*len(df)).agg([np.sum, np.mean, np.std, np.median,
                    np.var, np.min, np.max, percentile(5), percentile(25), 
                    percentile(50), percentile(75), percentile(95)])
    df_desc.reset_index(drop=True, inplace=True)
    df_desc.columns = [('_{}_'.format(prefix)).join(x).strip() for x in df_desc.columns.values]
    return df_desc

In [13]:
def dataset_info(df, entity_name, entity_id, numeric=None, factor=None, factor_num = 50):
    """
    will flatten a dataset based on numeric and factor variables
    factor variables will change to one-hot encoding.
    """
    df_info = []
    if numeric is None:
        newdf = dataframe_info(df.select_dtypes(include=[np.number]))
    else:
        newdf = dataframe_info(df[numeric])
        
    # factor information will one hot if unique number is less than 50
    if factor is None:
        factor = []
        for col in df.columns:
            if len(df[col].unique()) <= factor_num and not col.startswith(entity_name):
                factor.append(col)
    
    newfactor = dataframe_info(pd.get_dummies(df[factor]), 'factor')
    flatten_df = pd.concat([pd.DataFrame({entity_name:[entity_id]}), 
                            newdf, newfactor], axis=1)
    return flatten_df


In [14]:
list(dataset_info(atom[atom['atom_id'].isin(molecule_level_rship[0][1])].drop('atom_id', axis=1), 
             'molecule_id', 
            molecule_level_rship[0][0]).columns)

['molecule_id',
 'charge_num_sum',
 'charge_num_mean',
 'charge_num_std',
 'charge_num_median',
 'charge_num_var',
 'charge_num_amin',
 'charge_num_amax',
 'charge_num_percentile_5',
 'charge_num_percentile_25',
 'charge_num_percentile_50',
 'charge_num_percentile_75',
 'charge_num_percentile_95',
 'type_num_sum',
 'type_num_mean',
 'type_num_std',
 'type_num_median',
 'type_num_var',
 'type_num_amin',
 'type_num_amax',
 'type_num_percentile_5',
 'type_num_percentile_25',
 'type_num_percentile_50',
 'type_num_percentile_75',
 'type_num_percentile_95',
 'charge_factor_sum',
 'charge_factor_mean',
 'charge_factor_std',
 'charge_factor_median',
 'charge_factor_var',
 'charge_factor_amin',
 'charge_factor_amax',
 'charge_factor_percentile_5',
 'charge_factor_percentile_25',
 'charge_factor_percentile_50',
 'charge_factor_percentile_75',
 'charge_factor_percentile_95',
 'type_factor_sum',
 'type_factor_mean',
 'type_factor_std',
 'type_factor_median',
 'type_factor_var',
 'type_factor_amin'

In [15]:
# construct this for all columns...
molecule_atom_flat = pd.concat([
    dataset_info(atom[atom['atom_id'].isin(x[1])].drop('atom_id', axis=1), 
             'molecule_id', 
            x[0])
    for x in molecule_level_rship
])

In [16]:
molecule_atom_flat.head()

,charge_factor_amax,charge_factor_amin,charge_factor_mean,charge_factor_median,charge_factor_percentile_25,charge_factor_percentile_5,charge_factor_percentile_50,charge_factor_percentile_75,charge_factor_percentile_95,charge_factor_std,...,type_num_mean,type_num_median,type_num_percentile_25,type_num_percentile_5,type_num_percentile_50,type_num_percentile_75,type_num_percentile_95,type_num_std,type_num_sum,type_num_var
0,0.819,-0.381,-3.784851e-18,-0.080,-0.110,-0.36745,-0.080,0.1415,0.149,0.239584,...,19.045455,22.0,3.0,3.0,22.0,26.0,39.9,12.579186,419,158.235931
0,0.789,-0.411,0.000000e+00,0.090,-0.141,-0.38400,0.090,0.1200,0.120,0.238740,...,20.478261,22.0,3.0,3.0,22.0,26.0,40.0,14.048195,471,197.351779
0,0.835,-0.365,0.000000e+00,-0.094,-0.164,-0.33935,-0.094,0.1660,0.216,0.254246,...,32.208333,22.0,22.0,3.0,22.0,40.0,93.0,26.814952,773,719.041667
0,0.810,-0.540,1.434038e-17,-0.119,-0.389,-0.39000,-0.119,0.1410,0.810,0.400903,...,27.200000,25.5,22.0,3.0,25.5,40.0,40.0,12.777674,816,163.268966
0,0.807,-0.533,2.114711e-17,0.058,-0.122,-0.39300,0.058,0.1370,0.307,0.296773,...,18.047619,22.0,3.0,3.0,22.0,26.0,40.0,14.157953,379,200.447619


In [17]:
modelling_dataset = molecule.merge(molecule_atom_flat)

In [18]:
X = modelling_dataset.drop(['molecule_id', 'mutagenic'], axis=1)
# fill all na with mean - there will be nas if certain classes weren't in the dataset...
#X = X.apply(lambda x: x.fillna(x.mean()),axis=0) # for mean
X = X.fillna(0)

y = modelling_dataset['mutagenic'].tolist()

In [19]:
X_m = X.as_matrix()

In [20]:
model = make_pipeline(PCA(40), SGDClassifier())
model = SGDClassifier(penalty='l1')
kfold = KFold(n_splits=10, shuffle=False, random_state=42)
results = cross_val_score(model, X, y, cv=kfold)
print("Accuracy: {}".format(results.mean()))

Accuracy: 0.6178362573099416


In [21]:
model = SGDClassifier()
rfe = RFE(model, 40)
rfe = rfe.fit(X, y)

In [22]:
model.fit(X.as_matrix()[:, rfe.support_], y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [23]:
yhat = model.predict(X.as_matrix()[:, rfe.support_])
metric = accuracy_score(y, yhat)
print("log loss rate: {}".format(metric))

log loss rate: 0.648936170212766


In [24]:
from tpot import TPOTClassifier
pipeline_optimizer = TPOTClassifier()
#pipeline_optimizer.fit(X, y)